# Useful Functions for S&P and V&J



Into this Jupyter file we have **implemented some functions** used for *Sung and Poggio* and *Viola and Jones* models.
We have decided 

To import function from thise file to another jupyter file we have to import these modules:

In [1]:
import zipfile
import os
import datasets
import requests
import pandas as pd

This is the line code used from the jupyter files *Viola_Jones* and *Sung_Poggio* used to import functions from this file

In [8]:
#from ipynb.fs.defs.Useful_functions import "+ name of the function"

## Functions:

We have implemented a function that **draws red rectangles** on an image when a windows of fixed dimensions (19x19 for *Sung and Poggio* and 24x24 for *Viola and Jones*) gives a positive result for our trained model.

In [3]:
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import numpy as np


"""
Function that draw red rectangles on the image given a list of coordinates (x, y, h, w) and a PIL image object.
It returns an the same input images covered by a red bounding boxe drawn into the image, hopefully on the faces
"""
def draw_red_rectangles_on_image(image, coordinates):

    # Converting the image to a numpy array for plotting
    image_np = np.array(image)

    # Creating an ImageDraw object to draw on the photo
    draw = ImageDraw.Draw(image)

    # Defining the red color (R, G, B)
    red_color = (255, 0, 0)

    # Drawing the red rectangles on the image
    x, y, h, w = coordinates
    draw.rectangle([x, y, x + w, y + h], outline=red_color)

    # Converting the image back to PIL format
    image_with_rectangles = Image.fromarray(np.uint8(image_np))
    
    # Returning the new image with the red bounding boxes
    return image_with_rectangles

### Non-Maximum Suppression

The **Non-Maximum Suppression** (**NMS**) is a technique used for deleting the overlapping bounding boxes that gave positive results to the model. This technique is applied from both the *Viola and Jones* and *Sung and Poggio* models when 

In [4]:
import numpy as np

"""
This is a Non-Maximum Suppression (NMS) function to eliminate overlapping bounding boxes.

Arguments in input:
    coordinates_window (list): A list of bounding box coordinates in the format [x, y, w, h].
    overlapThresh (float, optional): The threshold value representing the minimum overlap required to consider two bounding boxes as the same object. Defaults to 0.8.

Returns:
    list: A new list of bounding boxes after NMS has been applied.
"""
def NMS(coordinates_window, overlapThresh):

    # Return an empty list if no bounding boxes are given
    if len(coordinates_window) == 0:
        return []

    # Extract the coordinates of bounding boxes
    x1 = np.array([box[0] for box in coordinates_window])
    y1 = np.array([box[1] for box in coordinates_window])
    x2 = np.array([box[0] + box[2] for box in coordinates_window])
    y2 = np.array([box[1] + box[3] for box in coordinates_window])
    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    indices = np.arange(len(x1))

    # Perform Non-Maximum Suppression
    for i, box in enumerate(coordinates_window):
        temp_indices = indices[indices != i]
        xx1 = np.maximum(box[0], x1[temp_indices])
        yy1 = np.maximum(box[1], y1[temp_indices])
        xx2 = np.minimum(box[0] + box[2], x2[temp_indices])
        yy2 = np.minimum(box[1] + box[3], y2[temp_indices])
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
        overlap = (w * h) / areas[temp_indices]

        if np.any(overlap > overlapThresh):
            # Remove the current box index (i) from the list of indices to suppress the current box
            indices = indices[indices != i]

    # Return the remaining non-overlapping bounding boxes
    return [coordinates_window[i] for i in indices]


We also implement a function that ***filters all the windows*** that have slided the image and that gave positive results from both the models.

In [5]:
"""
Function that creates a coordinated list which contains only the bounding boxes with the true label, given the coordinates of the window 
and the list of boolean labels
"""
def filter_boxes_by_boolean(coordinates_window, boolean_list):
    if len(coordinates_window) != len(boolean_list):
        raise ValueError("The input lists must have the same length.")
    
    boolean_array = np.array(boolean_list)
    filtered_boxes = np.array(coordinates_window)[boolean_array]
    
    return filtered_boxes.tolist()

We have implemented a function that ***merges the neighboring bounding boxes*** by a specified distance in order to reducing the number of final bounding boxes that maybe are selecting the same face but there could be some classification errors on the coordinates of the positive windows.

In [6]:
import numpy as np


"""
The merge_nearby_boxes function is intended to merge neighboring bounding boxes by a specified distance, 
creating resulting bounding boxes that enclose groups of adjacent or neighboring bounding boxes.
"""
def merge_nearby_boxes(bounding_boxes, mergeDistance):
    # Check if there are no bounding boxes
    if len(bounding_boxes) == 0:
        return []

    # Convert bounding_boxes to a NumPy array
    bounding_boxes = np.array(bounding_boxes)
    
    # Calculate the center of each bounding box
    box_centers = bounding_boxes[:, :2] + bounding_boxes[:, 2:] / 2
    
    # Calculate distances between box centers
    distances = np.linalg.norm(box_centers[:, np.newaxis, :] - box_centers, axis=2)

    # Set to keep track of merged indices
    merged_indices = set()
    merged_boxes = []

    # Iterate over each bounding box
    for i in range(len(bounding_boxes)):
        if i in merged_indices:
            continue

        # Find nearby bounding boxes
        nearby_indices = np.where((distances[i] < mergeDistance) & (~np.isin(np.arange(len(bounding_boxes)), list(merged_indices))))[0]
        
        if nearby_indices.size > 0:
            # Update merged indices
            merged_indices.update(nearby_indices)
            merged_indices.add(i)
            
            # Calculate the mean of nearby bounding boxes
            merged_box = np.mean(bounding_boxes[nearby_indices], axis=0)
            merged_boxes.append([int(coord) for coord in merged_box])
        else:
            # If no nearby bounding boxes, keep the current box
            merged_boxes.append([int(coord) for coord in bounding_boxes[i]])

    return merged_boxes